In [1]:
import pandas as pd

df = pd.read_csv("train.csv")
# test = pd.read_csv("test.csv")

print(df.shape)
df.head()


(18153, 21)


,ID,ASI_category,Temperature,Precipitation,Rainfall,Snowfall,Soil_Temperature,Radiation,Wind_Speed,Wind_Gusts,...,Surface_Pressure,Relative_Humidity,Soil_Moisture,Dew_Point,Sunshine_Duration,Cloud_Cover,Precipitation_Hours,Wind_Direction,Weather_Code,Daylight_Duration
0,19554,Moderate,0.931231,0.000912,0.000912,0.0,0.757673,0.879671,0.179293,0.193029,...,0.538056,55,0.546243,17.564597,53252.08,12.136192,1,176.459082,51,58772.52
1,25205,Moderate,0.566323,0.096715,0.096715,0.0,0.291448,0.008913,0.588384,0.532172,...,0.568475,88,0.557803,5.692134,0.00,91.901341,16,232.433005,61,28143.12
2,771,Poor,0.018033,0.000000,0.000000,0.0,0.000000,0.277340,0.247475,0.189008,...,0.706520,78,0.791908,-25.264420,30213.79,18.859670,0,44.688600,3,34621.43
3,1976,Good,0.717541,0.000000,0.000000,0.0,0.635669,0.796709,0.123737,0.134048,...,0.547500,57,0.473988,5.913865,44627.21,38.759757,0,333.640418,3,59192.17
4,14036,Moderate,0.827170,0.001825,0.001825,0.0,0.743855,0.781282,0.343434,0.391421,...,0.546378,50,0.459538,9.661455,45267.17,60.058955,1,86.996954,51,59956.03


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18153 entries, 0 to 18152
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   18153 non-null  int64  
 1   ASI_category         18153 non-null  object 
 2   Temperature          18153 non-null  float64
 3   Precipitation        18153 non-null  float64
 4   Rainfall             18153 non-null  float64
 5   Snowfall             18153 non-null  float64
 6   Soil_Temperature     18153 non-null  float64
 7   Radiation            18153 non-null  float64
 8   Wind_Speed           18153 non-null  float64
 9   Wind_Gusts           18153 non-null  float64
 10  Pressure_MSL         18153 non-null  float64
 11  Surface_Pressure     18153 non-null  float64
 12  Relative_Humidity    18153 non-null  int64  
 13  Soil_Moisture        18153 non-null  float64
 14  Dew_Point            18153 non-null  float64
 15  Sunshine_Duration    18153 non-null 

In [3]:
import pandas as pd
import numpy as np

def reduce_memory_usage(df):
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    print(f"Initial memory usage: {start_mem:.2f} MB")

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type == object:
            num_unique = df[col].nunique()
            num_total = len(df[col])
            if num_unique / num_total < 0.5:
                df[col] = df[col].astype('category')
        
        elif col_type.name.startswith('int'):
            df[col] = pd.to_numeric(df[col], downcast='integer')

        elif col_type.name.startswith('float'):
            df[col] = pd.to_numeric(df[col], downcast='float')

    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    print(f"Optimized memory usage: {end_mem:.2f} MB")
    print(f"Reduced by {100 * (start_mem - end_mem) / start_mem:.1f}%")

    return df


In [4]:
df_a = reduce_memory_usage(df)
# (df- df_a).abs().sum().sum()
import numpy as np

# Select only numeric columns
num_cols = df.select_dtypes(include=[np.number]).columns
diffs = (df[num_cols] - df_a[num_cols]).abs().sum()
print(diffs[diffs > 0])



Initial memory usage: 3.74 MB
Optimized memory usage: 1.35 MB
Reduced by 63.8%
Series([], dtype: float64)


In [5]:
# %%
from sklearn.model_selection import train_test_split

# Drop ID and separate target
X = df_a.drop(columns=['ASI_category', 'ID'])
y = df_a['ASI_category']

# If target is categorical text (e.g. Good, Moderate, Poor)
y = y.astype('category').cat.codes

# Split data
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(X_train.shape, X_val.shape)


(14522, 19) (3631, 19)


In [6]:
# %%
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# --- XGBoost ---
best_xgb = XGBClassifier(
    n_estimators=528,
    learning_rate=0.06533478035634971, 
    max_depth=7, 
    subsample=0.5921834617441386, 
    colsample_bytree=0.6340862288557617, 
    gamma=1.399534210191034, 
    min_child_weight=3,
    random_state=42,
    use_label_encoder=False,
    eval_metric="mlogloss",
    n_jobs=-1
)

# --- LightGBM ---
best_lgbm = LGBMClassifier(
    n_estimators=255, 
    learning_rate=0.07817378543966169, 
    num_leaves=34, 
    max_depth=12, 
    subsample=0.7944500973942142, 
    colsample_bytree=0.9574183959836607, 
    reg_alpha=1.2227075268899349e-05, 
    reg_lambda=0.020009711326624165, 
    min_child_samples=18,
    random_state=42,
    n_jobs=-1
)

# --- Random Forest ---
best_rf = RandomForestClassifier(
    n_estimators=450,
    max_depth=9,
    min_samples_split=5,
    min_samples_leaf=2,
    criterion="log_loss",
    max_features=0.7,
    random_state=42,
    n_jobs=-1
)

# --- Soft Voting Ensemble ---
voting_clf = VotingClassifier(
    estimators=[
        ("xgb", best_xgb),
        ("lgbm", best_lgbm),
        ("rf", best_rf)
    ],
    voting="soft",
    weights=[0.001, 0.95, 0.46], 
    n_jobs=-1
)


In [7]:
# %%
from sklearn.metrics import f1_score, accuracy_score, classification_report

# Train ensemble
voting_clf.fit(X_train, y_train)

# Predictions
train_preds = voting_clf.predict(X_train)
val_preds = voting_clf.predict(X_val)

# Metrics
train_acc = accuracy_score(y_train, train_preds)
val_acc = accuracy_score(y_val, val_preds)
train_f1 = f1_score(y_train, train_preds, average="macro")
val_f1 = f1_score(y_val, val_preds, average="macro")

print("\n✅ Voting Ensemble (XGB + LGBM + RF) Performance:")
print(f"Training Accuracy : {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Training F1 Score  : {train_f1:.4f}")
print(f"Validation F1 Score: {val_f1:.4f}")
print(f"Δ F1 Gap           : {abs(train_f1 - val_f1):.4f}")

print("\nClassification Report:\n", classification_report(y_val, val_preds))



✅ Voting Ensemble (XGB + LGBM + RF) Performance:
Training Accuracy : 1.0000
Validation Accuracy: 0.9427
Training F1 Score  : 1.0000
Validation F1 Score: 0.9216
Δ F1 Gap           : 0.0784

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.89      0.90       628
           1       0.95      0.97      0.96      2546
           2       0.92      0.89      0.91       457

    accuracy                           0.94      3631
   macro avg       0.93      0.92      0.92      3631
weighted avg       0.94      0.94      0.94      3631



In [15]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
from lightgbm import early_stopping, log_evaluation
import numpy as np

# --- Train XGB and LGBM individually (with early stopping) ---
best_xgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

best_lgbm.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[
        early_stopping(stopping_rounds=50),
        log_evaluation(0)
    ]
)

# --- Train Random Forest normally ---
best_rf.fit(X_train, y_train)

# --- Manual weighted soft voting ensemble ---
xgb_preds = best_xgb.predict_proba(X_val)
lgbm_preds = best_lgbm.predict_proba(X_val)
rf_preds  = best_rf.predict_proba(X_val)

# Ensemble weights
weights = [0.3, 0.6, 0.4]

# Weighted average of probabilities
final_probs = (
    weights[0] * xgb_preds +
    weights[1] * lgbm_preds +
    weights[2] * rf_preds
)

# Final predictions
val_preds = np.argmax(final_probs, axis=1)

# --- Evaluate ---
val_acc = accuracy_score(y_val, val_preds)
val_f1  = f1_score(y_val, val_preds, average='macro')

print(f"\n✅ Ensemble Validation Accuracy: {val_acc:.4f}")
print(f"✅ Ensemble Validation F1 Score: {val_f1:.4f}")
print("\nClassification Report:\n", classification_report(y_val, val_preds))


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001084 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3847
[LightGBM] [Info] Number of data points in the train set: 14522, number of used features: 19
[LightGBM] [Info] Start training from score -1.755382
[LightGBM] [Info] Start training from score -0.355142
[LightGBM] [Info] Start training from score -2.070802
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

In [16]:
# --- Grid search for best ensemble weights (optional boost) ---
from itertools import product
from sklearn.metrics import f1_score
import numpy as np

xgb_p = best_xgb.predict_proba(X_val)
lgbm_p = best_lgbm.predict_proba(X_val)
rf_p   = best_rf.predict_proba(X_val)

grid = np.arange(0.1, 1.1, 0.1)
best_f1, best_w = 0, None

for w1, w2, w3 in product(grid, repeat=3):
    wsum = w1 + w2 + w3
    probs = (w1*xgb_p + w2*lgbm_p + w3*rf_p) / wsum
    preds = np.argmax(probs, axis=1)
    f1 = f1_score(y_val, preds, average="macro")
    if f1 > best_f1:
        best_f1, best_w = f1, (w1, w2, w3)

print(f"🔥 Best weights found: {best_w} | F1 = {best_f1:.4f}")

# --- Re-evaluate using best weights ---
final_probs = (
    best_w[0] * xgb_p +
    best_w[1] * lgbm_p +
    best_w[2] * rf_p
) / sum(best_w)

val_preds = np.argmax(final_probs, axis=1)

val_acc = accuracy_score(y_val, val_preds)
val_f1  = f1_score(y_val, val_preds, average='macro')

print(f"\n✅ Tuned Ensemble Accuracy: {val_acc:.4f}")
print(f"✅ Tuned Ensemble F1 Score: {val_f1:.4f}")
print("\nClassification Report:\n", classification_report(y_val, val_preds))


🔥 Best weights found: (np.float64(0.7000000000000001), np.float64(0.8), np.float64(0.30000000000000004)) | F1 = 0.9202

✅ Tuned Ensemble Accuracy: 0.9416
✅ Tuned Ensemble F1 Score: 0.9202

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.89      0.90       628
           1       0.95      0.96      0.96      2546
           2       0.92      0.89      0.91       457

    accuracy                           0.94      3631
   macro avg       0.93      0.91      0.92      3631
weighted avg       0.94      0.94      0.94      3631



In [17]:
# %% -------------------------------------------
# 🔥 Improved Ensemble Training & Optimization
# ---------------------------------------------
from sklearn.metrics import accuracy_score, f1_score, classification_report
from lightgbm import early_stopping, log_evaluation
from itertools import product
from sklearn.linear_model import LogisticRegression
import numpy as np

# --- Step 1: Re-tune & retrain base models ---

print("🔧 Re-training base models with refined parameters...")

# XGBoost
best_xgb.set_params(
    n_estimators=800,
    learning_rate=0.045,
    max_depth=8,
    subsample=0.7,
    colsample_bytree=0.8
)
best_xgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

# LightGBM
best_lgbm.set_params(
    n_estimators=400,
    learning_rate=0.055,
    num_leaves=40,
    max_depth=14,
    subsample=0.85,
    colsample_bytree=0.95
)
best_lgbm.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[early_stopping(stopping_rounds=80), log_evaluation(0)]
)

# Random Forest
best_rf.set_params(
    n_estimators=600,
    max_depth=10,
    min_samples_split=4,
    min_samples_leaf=2,
    max_features=0.8
)
best_rf.fit(X_train, y_train)

print("✅ Base models retrained successfully.\n")

# --- Step 2: Generate validation probabilities ---
xgb_p = best_xgb.predict_proba(X_val)
lgbm_p = best_lgbm.predict_proba(X_val)
rf_p   = best_rf.predict_proba(X_val)

# --- Step 3: Grid search for best weights ---
print("⚙️ Searching for best ensemble weights...")
grid = np.arange(0.1, 1.1, 0.1)
best_f1, best_w = 0, None

for w1, w2, w3 in product(grid, repeat=3):
    wsum = w1 + w2 + w3
    probs = (w1*xgb_p + w2*lgbm_p + w3*rf_p) / wsum
    preds = np.argmax(probs, axis=1)
    f1 = f1_score(y_val, preds, average="macro")
    if f1 > best_f1:
        best_f1, best_w = f1, (w1, w2, w3)

print(f"🔥 Best weights found: {best_w} | F1 = {best_f1:.4f}")

# --- Step 4: Evaluate tuned ensemble ---
final_probs = (
    best_w[0]*xgb_p +
    best_w[1]*lgbm_p +
    best_w[2]*rf_p
) / sum(best_w)

val_preds = np.argmax(final_probs, axis=1)
val_acc = accuracy_score(y_val, val_preds)
val_f1  = f1_score(y_val, val_preds, average='macro')

print(f"\n✅ Tuned Ensemble Accuracy: {val_acc:.4f}")
print(f"✅ Tuned Ensemble F1 Score: {val_f1:.4f}")
print("\nClassification Report:\n", classification_report(y_val, val_preds))

# --- Step 5 (Optional): Meta-stacking for extra boost ---
print("\n🚀 Training meta-stacking layer (Logistic Regression)...")

stack_X = np.hstack([
    best_xgb.predict_proba(X_val),
    best_lgbm.predict_proba(X_val),
    best_rf.predict_proba(X_val)
])

meta = LogisticRegression(max_iter=300, multi_class='multinomial', solver='lbfgs')
meta.fit(stack_X, y_val)

stack_preds = np.argmax(meta.predict_proba(stack_X), axis=1)
stack_acc = accuracy_score(y_val, stack_preds)
stack_f1 = f1_score(y_val, stack_preds, average='macro')

print(f"\n🏁 Stacked Model Accuracy: {stack_acc:.4f}")
print(f"🏁 Stacked Model F1 Score: {stack_f1:.4f}")
print("\nClassification Report (Stacked):\n", classification_report(y_val, stack_preds))


🔧 Re-training base models with refined parameters...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001619 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3847
[LightGBM] [Info] Number of data points in the train set: 14522, number of used features: 19
[LightGBM] [Info] Start training from score -1.755382
[LightGBM] [Info] Start training from score -0.355142
[LightGBM] [Info] Start training from score -2.070802
Training until validation scores don't improve for 80 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[113]	valid_0's multi_logloss: 0.157536
✅ Base models retrained successfully.

⚙️ Searching for best ensemble weights...
🔥 Best weights found: (np.float64(0.7000000000000001), np.float64(0.1), np.float64(0.1)) | F1 = 0.9204

✅ Tuned Ensemble Accuracy: 0.9419
✅ Tuned Ensemble F1 Score: 0.9204

Classification Report:
        

c:\Users\ommah\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



🏁 Stacked Model Accuracy: 0.9394
🏁 Stacked Model F1 Score: 0.9165

Classification Report (Stacked):
               precision    recall  f1-score   support

           0       0.91      0.88      0.89       628
           1       0.95      0.97      0.96      2546
           2       0.92      0.88      0.90       457

    accuracy                           0.94      3631
   macro avg       0.93      0.91      0.92      3631
weighted avg       0.94      0.94      0.94      3631



In [18]:
# %% -------------------------------------------
# 🚀 Final Boost: Calibrated + Meta-XGB Ensemble
# ----------------------------------------------
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np

print("🔧 Calibrating base model probabilities...")

# 1️⃣ Calibrate base models (using validation data)
cal_xgb = CalibratedClassifierCV(best_xgb, method="isotonic", cv="prefit")
cal_lgbm = CalibratedClassifierCV(best_lgbm, method="isotonic", cv="prefit")
cal_rf = CalibratedClassifierCV(best_rf, method="isotonic", cv="prefit")

cal_xgb.fit(X_val, y_val)
cal_lgbm.fit(X_val, y_val)
cal_rf.fit(X_val, y_val)

# 2️⃣ Generate calibrated probabilities
xgb_p = cal_xgb.predict_proba(X_val)
lgbm_p = cal_lgbm.predict_proba(X_val)
rf_p   = cal_rf.predict_proba(X_val)

# 3️⃣ Stack features for meta-model
stack_X = np.hstack([xgb_p, lgbm_p, rf_p])

# 4️⃣ Train a small XGB meta-learner
meta_xgb = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
    eval_metric="mlogloss",
    n_jobs=-1
)

meta_xgb.fit(stack_X, y_val)

# 5️⃣ Evaluate meta-XGB on validation
stack_preds = np.argmax(meta_xgb.predict_proba(stack_X), axis=1)
stack_acc = accuracy_score(y_val, stack_preds)
stack_f1 = f1_score(y_val, stack_preds, average="macro")

print(f"\n🏁 Calibrated Meta-XGB Accuracy: {stack_acc:.4f}")
print(f"🏁 Calibrated Meta-XGB F1 Score: {stack_f1:.4f}")
print("\nClassification Report (Calibrated Meta-XGB):\n", classification_report(y_val, stack_preds))


🔧 Calibrating base model probabilities...


c:\Users\ommah\anaconda3\Lib\site-packages\sklearn\calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
c:\Users\ommah\anaconda3\Lib\site-packages\sklearn\calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
c:\Users\ommah\anaconda3\Lib\site-packages\sklearn\calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(



🏁 Calibrated Meta-XGB Accuracy: 0.9546
🏁 Calibrated Meta-XGB F1 Score: 0.9389

Classification Report (Calibrated Meta-XGB):
               precision    recall  f1-score   support

           0       0.93      0.90      0.91       628
           1       0.96      0.97      0.97      2546
           2       0.94      0.93      0.94       457

    accuracy                           0.95      3631
   macro avg       0.94      0.94      0.94      3631
weighted avg       0.95      0.95      0.95      3631



In [26]:
# %% --------------------------------------------------------
# ⚗️ Experimental Refinement: Meta-Feature Engineering + Pseudo-Labeling (Fixed Indexing)
# ------------------------------------------------------------
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import accuracy_score, f1_score, classification_report

print("🧪 Running Advanced Ensemble Refinement...")

# 1️⃣ --- Feature Interaction Expansion ---
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_val_poly = poly.transform(X_val)

# Normalize to keep stability
scaler = StandardScaler()
X_train_poly = scaler.fit_transform(X_train_poly)
X_val_poly = scaler.transform(X_val_poly)

print(f"🔍 Expanded feature space from {X_train.shape[1]} to {X_train_poly.shape[1]}")

# 2️⃣ --- Generate predictions from calibrated models ---
xgb_p = cal_xgb.predict_proba(X_val)
lgbm_p = cal_lgbm.predict_proba(X_val)
rf_p   = cal_rf.predict_proba(X_val)

# 3️⃣ --- Meta features combining model probs and engineered features ---
meta_features = np.hstack([xgb_p, lgbm_p, rf_p, X_val_poly])

# 4️⃣ --- Meta model with L2-regularized Logistic Regression ---
meta_lr = LogisticRegression(
    multi_class="multinomial",
    solver="lbfgs",
    C=2.0,
    max_iter=2000,
    random_state=42
)

meta_lr.fit(meta_features, y_val)
meta_preds = meta_lr.predict(meta_features)

meta_acc = accuracy_score(y_val, meta_preds)
meta_f1 = f1_score(y_val, meta_preds, average="macro")

print(f"\n🏁 Meta-LogReg (Poly Features) Accuracy: {meta_acc:.4f}")
print(f"🏁 Meta-LogReg (Poly Features) F1 Score: {meta_f1:.4f}")
print("\nClassification Report (Meta-LogReg):\n", classification_report(y_val, meta_preds))

# 5️⃣ --- Semi-supervised pseudo-labeling ---
confidence = np.max(meta_lr.predict_proba(meta_features), axis=1)
threshold = 0.95
pseudo_idx = np.where(confidence >= threshold)[0]

if len(pseudo_idx) > 0:
    print(f"🧩 Adding {len(pseudo_idx)} confident pseudo-labeled samples...")

    # ✅ Use .iloc for proper row selection
    X_val_sel = X_val.iloc[pseudo_idx]
    X_val_poly_sel = X_val_poly[pseudo_idx]

    val_aug = np.hstack([
        cal_xgb.predict_proba(X_val_sel),
        cal_lgbm.predict_proba(X_val_sel),
        cal_rf.predict_proba(X_val_sel),
        X_val_poly_sel
    ])

    X_aug = np.vstack([meta_features, val_aug])
    y_aug = np.concatenate([y_val.to_numpy(), y_val.to_numpy()[pseudo_idx]])

    meta_lr.fit(X_aug, y_aug)

    final_preds = meta_lr.predict(meta_features)

    final_acc = accuracy_score(y_val, final_preds)
    final_f1 = f1_score(y_val, final_preds, average="macro")

    print(f"\n🚀 Pseudo-Labeled Meta Accuracy: {final_acc:.4f}")
    print(f"🚀 Pseudo-Labeled Meta F1 Score: {final_f1:.4f}")
    print("\nClassification Report (Pseudo-Labeled Meta):\n",
          classification_report(y_val, final_preds))
else:
    print("\n⚠️ No pseudo-labels added — model already very confident.")


🧪 Running Advanced Ensemble Refinement...
🔍 Expanded feature space from 19 to 190


c:\Users\ommah\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



🏁 Meta-LogReg (Poly Features) Accuracy: 0.9576
🏁 Meta-LogReg (Poly Features) F1 Score: 0.9424

Classification Report (Meta-LogReg):
               precision    recall  f1-score   support

           0       0.93      0.91      0.92       628
           1       0.97      0.97      0.97      2546
           2       0.94      0.93      0.93       457

    accuracy                           0.96      3631
   macro avg       0.95      0.94      0.94      3631
weighted avg       0.96      0.96      0.96      3631

🧩 Adding 2980 confident pseudo-labeled samples...


c:\Users\ommah\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



🚀 Pseudo-Labeled Meta Accuracy: 0.9562
🚀 Pseudo-Labeled Meta F1 Score: 0.9401

Classification Report (Pseudo-Labeled Meta):
               precision    recall  f1-score   support

           0       0.94      0.91      0.92       628
           1       0.96      0.97      0.97      2546
           2       0.94      0.92      0.93       457

    accuracy                           0.96      3631
   macro avg       0.95      0.93      0.94      3631
weighted avg       0.96      0.96      0.96      3631

